In [ ]:
# -*- coding: utf-8 -*-
# <nbformat>4.1</nbformat>

# <codecell>

import pandas as pd
import numpy as np

# Configuration pour afficher toutes les colonnes des DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Définition du chemin du fichier de données
DATA_PATH = '../app/data/raw/online_retail_ii.xlsx' # Assurez-vous d'avoir le fichier dans ce chemin

# Chargement du jeu de données (Online Retail II)
try:
    # Lecture du fichier Excel. Assurez-vous d'utiliser le bon nom de fichier (ici .xlsx)
    df = pd.read_excel(DATA_PATH)
    print("Données chargées avec succès.")
except FileNotFoundError:
    print(f"ERREUR : Le fichier de données est introuvable à l'adresse {DATA_PATH}.")
    print("Veuillez télécharger le jeu de données 'Online Retail II' et le placer dans 'app/data/raw/'.")
    df = pd.DataFrame() # Crée un DataFrame vide en cas d'erreur
    
# Affichage des 5 premières lignes pour un aperçu rapide
if not df.empty:
    display(df.head())

# <markdown>
# # Partie I : Notebook d'Exploration Visuelle Complète
# 
# L'objectif de cette première phase est d'acquérir une compréhension exhaustive du jeu de données *Online Retail II* (UCI) afin de cadrer les analyses et la construction de l'application Streamlit.
# 
# ---
# 
# ## Fiche Synthétique des Données
# 
# ### 1. Source, Volume et Période

# <codecell>

if not df.empty:
    # 1. Volume et Période
    print(f"Source: Online Retail II (UCI)")
    print(f"Volume: {df.shape[0]:,} lignes et {df.shape[1]} colonnes")
    
    # Conversion de la colonne d'horodatage si nécessaire (assumant qu'elle est déjà en datetime si l'on utilise read_excel)
    # df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']) 
    
    min_date = df['InvoiceDate'].min()
    max_date = df['InvoiceDate'].max()
    
    print(f"Période couverte : Du {min_date.strftime('%d/%m/%Y')} au {max_date.strftime('%d/%m/%Y')}")
    print(f"Clients uniques : {df['Customer ID'].nunique():,} (excluant les valeurs manquantes)")

# <markdown>
# ### 2. Dictionnaire des Variables (À Compléter)
# 
# Remplissez ce tableau avec les détails de chaque colonne.
# 
# | Nom de la Variable | Type (Python) | Sémantique | Unités/Valeurs | Colonne Importante? |
# |:------------------|:--------------|:-----------|:---------------|:-------------------|
# | `InvoiceNo`       | object        | Numéro de facture. Commence par 'C' si annulation (retour). | Entier/Chaîne de caractères | Oui |
# | `StockCode`       | object        | Code produit (Item). | Chaîne de caractères | Oui |
# | `Description`     | object        | Nom du produit. | Chaîne de caractères | Oui |
# | `Quantity`        | int64         | Quantité de produits achetés par transaction. | Nombre entier | Oui |
# | `InvoiceDate`     | datetime64[ns]| Date et heure de la transaction. | Horodatage | Oui |
# | `Price`           | float64       | Prix unitaire du produit. | Monétaire (GBP, assumé) | Oui |
# | `Customer ID`     | float64       | Identifiant unique du client. | Entier (float avec NaN) | Oui |
# | `Country`         | object        | Pays de destination de la commande. | Chaîne de caractères | Oui |
# 
# ## Qualité des Données
# 
# ### 3. Valeurs Manquantes

# <codecell>

if not df.empty:
    print("Aperçu des valeurs manquantes par colonne (en %):")
    missing_data = df.isnull().sum() / len(df) * 100
    display(missing_data[missing_data > 0].sort_values(ascending=False))
    
    # Notez l'importance des ID clients manquants pour l'analyse CLV/Cohortes
    nan_customers_count = df['Customer ID'].isnull().sum()
    print(f"\nTotal des lignes sans Customer ID : {nan_customers_count:,} ({missing_data['Customer ID']:.2f} % des données).")

# <markdown>
# ### 4. Règles d'Annulation (Retours) et Outliers
# 
# **Règle :** Les factures (`InvoiceNo`) commençant par "C" sont des annulations ou des retours.

# <codecell>

if not df.empty:
    # Identifier les retours
    df['is_return'] = df['InvoiceNo'].astype(str).str.startswith('C')
    return_count = df['is_return'].sum()
    
    print(f"Nombre total de transactions : {df.shape[0]:,}")
    print(f"Nombre de retours (lignes 'C...') : {return_count:,} ({return_count/df.shape[0]*100:.2f} % du total)")

    # Vérification des outliers sur Quantity et Price
    print("\nStatistiques sur les quantités et les prix (incluant les retours/annulations):")
    display(df[['Quantity', 'Price']].describe().T)

    # Filtrer les quantités ou prix nuls/négatifs (autres que les retours marqués 'C')
    negative_quantity = df[df['Quantity'] < 0].shape[0]
    zero_price = df[df['Price'] <= 0].shape[0]

    print(f"\nLignes avec Quantity < 0 (hors 'C'): {negative_quantity:,}")
    print(f"Lignes avec Price <= 0: {zero_price:,}")
    
    print("\n--- Poursuivre avec la suppression des doublons et la création des premières métriques (ex: TotalPrice) ---")

# <markdown>
# ## Prochaine Étape : Nettoyage et Création de Métriques
# 
# Une fois cette exploration terminée, l'étape logique est :
# 1. **Nettoyer** : Supprimer les lignes avec ID client manquant.
# 2. **Prétraiter** : Gérer les retours (les neutraliser, les exclure, ou les utiliser, selon l'approche souhaitée).
# 3. **Calculer** : Créer la colonne `TotalRevenue` (`Quantity` * `Price`).
# 4. **Visualiser** : Créer les 6-8 graphiques demandés.

# <codecell>
# Fin du Notebook initial.

ERREUR : Le fichier de données est introuvable à l'adresse ../app/data/raw/online_retail_ii.xlsx.
Veuillez télécharger le jeu de données 'Online Retail II' et le placer dans 'app/data/raw/'.
